<b><div style="text-align: center;"><b>Проект по SQL</b></div></b><br>

<b><div style="text-align: center;">Описание проекта</div></b><br>
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.<br>
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.<br>
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.<br>

<b><div style="text-align: center;">Задача проекта</div></b><br>
Проанализировать базу данных о книгах и дать рекомендации по формированию подборки для клиентов сервиса.<br><br>

<b><div style="text-align: center;">Описание данных</div></b><br>
Таблица <b>books</b> содержит данные о книгах:<br>
book_id — идентификатор книги;<br>
author_id — идентификатор автора;<br>
title — название книги;<br>
num_pages — количество страниц;<br>
publication_date — дата публикации книги;<br>
publisher_id — идентификатор издателя.<br>
Таблица <b>authors</b> содержит данные об авторах:<br>
author_id — идентификатор автора;<br>
author — имя автора.<br>
Таблица <b>publishers</b> содержит данные об издательствах:<br>
publisher_id — идентификатор издательства;<br>
publisher — название издательства;<br>
Таблица <b>ratings</b> содержит данные о пользовательских оценках книг:<br>
rating_id — идентификатор оценки;<br>
book_id — идентификатор книги;<br>
username — имя пользователя, оставившего оценку;<br>
rating — оценка книги.<br>
Таблица <b>reviews</b> содержит данные о пользовательских обзорах:<br>
review_id — идентификатор обзора;<br>
book_id — идентификатор книги;<br>
username — имя автора обзора;<br>
text — текст обзора.<br><br>

<b><div style="text-align: center;">Схема данных</div></b><br>
<img src="https://pictures.s3.yandex.net/resources/scheme_1589269096.png" alt="схема данных">

<b>Декомпозиция</b><br>
<a href='#section1'>1.Предобработка данных</a><br>
<a href='#section2'>2.Исследовальский анализ данных</a><br>
<a href='#section21'>2.1 Посчитайте, сколько книг вышло после 1 января 2000 года</a><br>
<a href='#section22'>2.2 Для каждой книги посчитайте количество обзоров и среднюю оценку</a><br>
<a href='#section23'>2.3 Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры</a><br>
<a href='#section24'>2.4 Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками</a><br>
<a href='#section25'>2.5 Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.</a><br>
<a href='#section3'>3. Выводы и рекомндации</a><br>

<a id='section1'></a><br>
## Предобработка данных

In [36]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
pd.options.display.max_colwidth=400
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'xxxxx', # пароль
             'host': 'xxxx',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                 db_config['pwd'],
                                                 db_config['host'],
                                                 db_config['port'],
                                                 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Выведем основную информацию и заголовок таблиц, для этого создим функцию и вызовем ее в цикле

In [37]:
db = ['books', 'authors', 'publishers', 'ratings', 'reviews']
def db_inf(db):
    query = '''
            SELECT *
            FROM {}
         '''.format(db)
    info=pd.io.sql.read_sql(query, con = engine)
    display(info.head())
    print(info.info())
    print('-'*50)
    
for i in range (0,len(db)):
    print('Таблица:',db[i])
    db_inf(db[i])

Таблица: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None
--------------------------------------------------
Таблица: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None
--------------------------------------------------
Таблица: publishers


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None
--------------------------------------------------
Таблица: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None
--------------------------------------------------
Таблица: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None
--------------------------------------------------


На этом этапе мы загрузили и проанализировали таблицы БД:<br>
- таблица books(количество книг в базе) содержит 6 полей и 1 000 записей (количество книг в базе), пропусков нет<br>
- таблица authors(количество авторов) содержит 2 поля и 636 записей (количество авторов), пропусков нет<br>
- таблица publishers(количество издательств) содержит 2 поля и 340 записей , пропусков нет<br>
- таблица ratings(данные о пользовательских оценках книг) содержит 4 поля и 6 456 записей, пропусков нет<br>
- таблица reviews(содержит данные о пользовательских обзорах) содержит 4 поля и 2 793 записи, пропусков нет<br>
Из данного анализа видно, что у нас всего 1000 книг, 636 авторов, 340 издательств. Всего 6456 пользовательских оценок и 2793 обзора.<br>
Далее проведем исследовальскийанализ данных, чтобы ответить на вопросы исследования.

<a id='section2'></a>
## Исследовальский анализ данных

<a id='section21'></a> 
### Расчет кол-ва книг, вышедших после 1 января 2000 года

In [38]:
query_1 = '''
            SELECT COUNT (book_id) AS cnt
            FROM books
            WHERE publication_date > '2000-01-01';
         '''
 
answer1 = pd.io.sql.read_sql(query_1, con = engine)
print('Таблица содержит данные о',answer1['cnt'][0],'книгах, вышедших после 2000 года')

Таблица содержит данные о 819 книгах, вышедших после 2000 года


Из представленых в базе 1000 книг, 819 вышли после 2000 года. Можно сделать вывод, что база состоит из современных книг.

<a id='section22'></a> 
###  Для каждой книги посчитайте количество обзоров и среднюю оценку

In [39]:
query_2 = '''
            SELECT b.book_id, title,
            COUNT(DISTINCT rw.review_id) AS review_count, ROUND(AVG(r.rating),2) AS avg_rating
            FROM books AS b LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
            LEFT JOIN ratings AS r ON rw.book_id = r.book_id
            GROUP BY b.book_id
            ORDER BY review_count DESC;            
         '''
answer2 = pd.io.sql.read_sql(query_2, con = engine)
display(answer2)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,NaN
996,808,The Natural Way to Draw,0,NaN
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,NaN
998,221,Essential Tales and Poems,0,NaN


In [40]:
print('max рейтинг', answer2['avg_rating'].max(),'max число review',answer2['review_count'].max())

max рейтинг 5.0 max число review 7


Данные этого запроса можно использовать для формирования ценностного предложения сервиса. Минимальный показатель книг - отстутствие рейтинга и 0 обзоров, мах число обзоров 7, мах рейтиг 5. Книги без отзывыв вряд ли будут пользоваться большой популярностью, лучше включать в состав предложения книги с рейтингом более 3 и числом обзоров более 2х.


<a id='section23'></a> 
### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [41]:
query_3 = '''
            SELECT p.publisher,
            COUNT(b.book_id) AS books_cnt
            FROM books AS b FULL JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY p.publisher
            ORDER BY books_cnt DESC
            LIMIT 1;            
         '''

answer3 = pd.io.sql.read_sql(query_3, con = engine)
display(answer3.head(10))
print('Больше всего книг выпустило издательство',answer3['publisher'][0],'оно выпустило',answer3['books_cnt'][0],'книги')

,publisher,books_cnt
0,Penguin Books,42


Больше всего книг выпустило издательство Penguin Books оно выпустило 42 книги


<a id='section24'></a> 
### Определите автора с самой высокой средней оценкой книг - учитывайте только книги с 50 и более оценками

In [42]:
query_4 = '''
            SELECT a.author,
            ROUND(AVG(r.rating),2) AS rating_avg
            FROM books AS b LEFT JOIN ratings AS r ON b.book_id = r.book_id
            LEFT JOIN authors As a ON b.author_id = a.author_id
            WHERE b.book_id IN (
                                SELECT book_id
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(rating_id) >= 50
                                )
            GROUP BY a.author
            ORDER BY rating_avg DESC
            LIMIT 1;
         '''

answer4 = pd.io.sql.read_sql(query_4, con = engine)
display(answer4)
print('Автор с самой высокой средней оценкой книг',answer4['author'][0],'средний рейтинг его книг',answer4['rating_avg'][0])

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг J.K. Rowling/Mary GrandPré средний рейтинг его книг 4.29


<a id='section25'></a> 
### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [43]:
query_5 = '''
            SELECT ROUND(AVG(rev_count),2) AS avg
            FROM (
            SELECT username,
            COUNT(DISTINCT review_id) AS rev_count
            FROM reviews
            WHERE username IN (SELECT username
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(DISTINCT rating_id) > 50)
            GROUP BY username) AS rw;
         '''

answer5 = pd.io.sql.read_sql(query_5, con = engine)
display(answer5)
print('Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок=',answer5['avg'][0])

,avg
0,24.33


Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок= 24.33


Данная цифра говорит о высокой увлеченности пользовталей и оценкам/обзорам пользоватлей, поставивших более 50 оценок вполне можно доверять и описаться на их мнение при выборе книг для сервиса.

<a id='section3'></a> 
## Рекомендации

1.Из представленых в базе 1000 книг, 819 вышли после 2000 года. База довольно актуальная. <br>
2.Было расчитано количество обзоров и средняя оценка, результат сохранен в датасает answer2. Эту информацию можно использовать для отбора потенциально успешных у читателей книг. Мах число обзоров 7, мах рейтиг 5. Книги без отзывыв вряд ли будут пользоваться большой популярностью, лучше включать в состав предложения книги с рейтингом более 3 и числом обзоров более 2х.<br>
3.Лидером среди издательств является Penguin Books оно выпустило 42 книги. Учитывая большую активность издательства, с ним можно посотрудничать в дальнейшем для расширения ассортиметнта книг по мере выхода новых изданий.<br>
4.Автор с самой высокой средней оценкой книг J.K. Rowling/Mary GrandPré средний рейтинг его книг 4.29. Книги этого автора обязательно следует включить в состав предложения сервиса.<br>
5.Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок= 24.33. Цифра говорит о высокой увлеченности и активности этих пользователей. Их оценкам волне можно доверять. Можно предложить таким пользоватлям какие нибудь преимущества в пользовании сервисом, для увеличения объективных обзоров и оценок, что в целом повлечет улучшение ценностного предложения сервиса.<br><br>

Был проведен поверхностный анализ, для более подробного пониманя состава книг неплохо бы еще провести исследование планируемой аудитории сервиса, возможно некоторые пользователи старшего возраста будут искать книги изданные до 2000 года, а их в представленной базе менее 20%.